In [11]:
import os
import numpy as np
import pandas as pd

In [12]:
df = pd.read_csv("/Users/admin/DataScience/midterm/student/student-mat.csv")
df.shape

(395, 33)

In [13]:
df = df.drop(columns = 'G2')
df = df.drop(columns = 'G1')
df.shape
df['sex'].replace(['M', 'F'], [1, 0], inplace=True)
df['address'].replace(['U', 'R'], [1, 0], inplace=True)
df['famsize'].replace(['LE3', 'GT3'], [1, 0], inplace=True)
df['Pstatus'].replace(['T', 'A'], [1, 0], inplace=True)
df['Mjob'].replace(['other', 'services','at_home','teacher','health'], [4,3,2,1,0], inplace=True)
df['Fjob'].replace(['other', 'services','at_home','teacher','health'], [4,3,2,1,0], inplace=True)
df['reason'].replace(['course', 'home','reputation','other'], [3,2,1,0], inplace=True)
df['schoolsup'].replace(['yes', 'no'], [1, 0], inplace=True)
df['famsup'].replace(['yes', 'no'], [1, 0], inplace=True)
df['paid'].replace(['yes', 'no'], [1, 0], inplace=True)
df['activities'].replace(['yes', 'no'], [1, 0], inplace=True)
df['nursery'].replace(['yes', 'no'], [1, 0], inplace=True)
df['higher'].replace(['yes', 'no'], [1, 0], inplace=True)
df['internet'].replace(['yes', 'no'], [1, 0], inplace=True)
df['romantic'].replace(['yes', 'no'], [1, 0], inplace=True)
df.loc[df['G3'] < 11, 'G3'] = 0
df.loc[df['G3'] > 10, 'G3'] = 1
X = df.iloc[:,1:30].values
y = df.iloc[:,-1].values

In [14]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 10] = labelencoder_X_1.fit_transform(X[:, 10])
onehotencoder = OneHotEncoder(categorical_features = [10])
X = onehotencoder.fit_transform(X).toarray()

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [15]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
random_state=30434)
mScaler =MinMaxScaler(copy=True, feature_range=(0, 1))
X_train = mScaler.fit_transform(X_train)
X_test = mScaler.transform(X_test)

In [16]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)
rfe = RFE(estimator=logisticRegr, n_features_to_select=10)
rfe = rfe.fit(X_train, y_train)

In [17]:
from sklearn.metrics import accuracy_score, precision_score,recall_score, f1_score
X_train_model = rfe.transform(X_train)
X_test_model = rfe.transform(X_test)
logisticRegr = LogisticRegression(solver='lbfgs', max_iter=1000)
logisticRegr.fit(X_train_model, y_train)
predictions = logisticRegr.predict(X_test_model)
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
print("Performance Logistic Regression:")
print(" - accuracy = " + str(accuracy))
print(" - precision = " + str(precision))
print(" - recall = " + str(recall))
print(" - f1 = " + str(f1))

Performance Logistic Regression:
 - accuracy = 0.569620253164557
 - precision = 0.52
 - recall = 0.7222222222222222
 - f1 = 0.6046511627906976


In [9]:
# Random forest
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators = 100, random_state=30434)
clf.fit(X_train_model,y_train)
predictions=clf.predict(X_test_model)
from sklearn.metrics import accuracy_score, precision_score,recall_score, f1_score
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
print("Performance Random forest:")
print(" - accuracy = " + str(accuracy))
print(" - precision = " + str(precision))
print(" - recall = " + str(recall))
print(" - f1 = " + str(f1))

Performance Random forest:
 - accuracy = 0.5569620253164557
 - precision = 0.5121951219512195
 - recall = 0.5833333333333334
 - f1 = 0.5454545454545454


In [18]:
# XGBOOT
import xgboost as xgb
gbm = xgb.XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05).fit(X_train_model, y_train)
predictions = gbm.predict(X_test_model)
from sklearn.metrics import accuracy_score, precision_score,recall_score, f1_score
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
print("Performance XGBOOT:")
print(" - accuracy = " + str(accuracy))
print(" - precision = " + str(precision))
print(" - recall = " + str(recall))
print(" - f1 = " + str(f1))

Performance XGBOOT:
 - accuracy = 0.5949367088607594
 - precision = 0.5476190476190477
 - recall = 0.6388888888888888
 - f1 = 0.5897435897435898


In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense
classifier = Sequential()
 # Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 10))
# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# Fitting ANN to the training set
classifier.fit(X_train_model, y_train, batch_size = 10, nb_epoch = 100)
# predictions = classifier.predict(X_test_model)
score = classifier.evaluate(X_test_model, y_test, batch_size=128)
print("Performance Neuron network:")
print("Test score:", score[0])
print("Test accuracy:", score[1])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=10, units=6, kernel_initializer="uniform")`
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/100
316/316 [==============================] - 0s 942us/step - loss: 0.6931 - acc: 0.5095
Epoch 2/100
316/316 [==============================] - 0s 124us/step - loss: 0.6924 - acc: 0.5475
Epoch 3/100
316/316 [==============================] - 0s 133us/step - loss: 0.6917 - acc: 0.5475
Epoch 4/100
316/316 [==============================] - 0s 122us/step - loss: 0.6905 - acc: 0.5475
Epoch 5/100
316/316 [==============================] - 0s 131us/step - loss: 0.6889 - acc: 0.5475
Epoch 6/100
316/316 [==============================] - 0s 138us/step - loss: 0.6859 - acc: 0.5475
Epoch 7/100
316/316 [==============================] - 0s 131us/step - loss: 0.6828 - acc: 0.5475
Epoch 8/100
316/316 [==============================] - 0s 128us/step - loss: 0.6798 - acc: 0.5475
Epoch 9/100
316/316 [==============================] - 0s 134us/step - loss: 0.6765 - acc: 0.5475
Epoch 10/100
316/316 [==============================] - 0s 145us/step - loss: 0.6739 - acc: 0.5475
Epoch 11/100
316/31